# Football Scheduler

Here we implement the model descripted at [[1]](#References) and run some test with real teams.

## Imports

In [1]:
from os import makedirs
import json
import random
import pandas as pd
from typing import Callable, Tuple, List
from IPython.display import display, Markdown

from model import FootballSchedulerModel, SymetricScheme
from parse import parse_sol, to_df, to_df_mapped

In [2]:
countries = ["ARG", "BOL", "BRA", "CHI", "COL", "ECU", "PAR", "PER", "URU", "VEN"]
top_countries = ["ARG", "BRA"]
assert(set(top_countries).issubset(set(countries)))
n = len(countries)

In [3]:
def raffle(countries: list[str]) -> Tuple[Callable[[str], int], Callable[[int], str]]:
	countries_copy = countries.copy()
	random.shuffle(countries_copy)
	return lambda c: countries_copy.index(c), lambda i: countries_copy[i]

def write_raffle(base_path: str, country_of: Callable[[int], str]):
	with open(f"{base_path}/country_of.json", "w") as f:
		json.dump(dict([(i, country_of(i)) for i in range(n)]), f)

# Represents the random asignment of country <-> id.
index_of, country_of = raffle(countries)

In [4]:
top_teams = [index_of(c) for c in top_countries]

In [5]:
def write_model_and_raffle(
	base_path: str, model: FootballSchedulerModel
):
	makedirs(base_path, exist_ok=True)
	write_raffle(base_path, country_of=country_of)
	model.write_problem(f"{base_path}/model.lp")


def assert_optimized(model: FootballSchedulerModel):
	model.optimize()
	assert model.get_obj_value() == 0

def write_solution(base_path: str, model: FootballSchedulerModel):
    sol_path = f"{base_path}/solution.sol"
    model.write_sol(sol_path)


def recover_solution_as_df(base_path: str, n: int) -> pd.DataFrame:
    solution_path = f"{base_path}/solution.sol"

    sol = parse_sol(solution_path)

    return to_df(
		n = n,
        sol=sol
    )


def recover_solution_as_mapped_df(base_path: str) -> pd.DataFrame:
	solution_path = f"{base_path}/solution.sol"
	country_of_path = f"{base_path}/country_of.json"

	sol = parse_sol(solution_path)
	country_of = json.load(open(country_of_path))

	return to_df_mapped(
		sol=sol,
		country_of=country_of,
	)

def write_solution_to_latex(base_path: str, sol_df: pd.DataFrame):
	with open(f"{base_path}/solution_table.tex", "w+") as f:
		f.write(sol_df.to_latex(index=False))

## French Scheme

In [6]:
base_path_french = "output/french"

In [7]:
french_model = FootballSchedulerModel(n, SymetricScheme.FRENCH, top_teams)
write_model_and_raffle(base_path_french, french_model)
assert_optimized(french_model)
print(
    f"Optimization of the french model with top teams took: {french_model.get_solving_time()} seconds"
)
write_solution(base_path_french, french_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/french/model.lp
Optimization of the french model with top teams took: 10.723665 seconds


In [8]:
french_df = recover_solution_as_mapped_df(base_path_french)
write_solution_to_latex(base_path_french, french_df)
display(Markdown(french_df.to_markdown(index=False)))

| Team   | 0    | 1    | 2    | 3    | 4    | 5    | 6    | 7    | 8    | 9    | 10   | 11   | 12   | 13   | 14   | 15   | 16   | 17   |
|:-------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
| PER    | VEN  | @BRA | @CHI | URU  | @ECU | COL  | @PAR | ARG  | @BOL | BRA  | CHI  | @URU | ECU  | @COL | PAR  | @ARG | BOL  | @VEN |
| CHI    | PAR  | @ECU | PER  | @COL | ARG  | @BOL | @VEN | URU  | @BRA | ECU  | @PER | COL  | @ARG | BOL  | VEN  | @URU | BRA  | @PAR |
| BRA    | @URU | PER  | COL  | @PAR | VEN  | @ARG | ECU  | @BOL | CHI  | @PER | @COL | PAR  | @VEN | ARG  | @ECU | BOL  | @CHI | URU  |
| COL    | BOL  | @VEN | @BRA | CHI  | URU  | @PER | @ARG | PAR  | @ECU | VEN  | BRA  | @CHI | @URU | PER  | ARG  | @PAR | ECU  | @BOL |
| VEN    | @PER | COL  | BOL  | @ARG | @BRA | PAR  | CHI  | @ECU | URU  | @COL | @BOL | ARG  | BRA  | @PAR | @CHI | ECU  | @URU | PER  |
| ECU    | @ARG | CHI  | PAR  | @BOL | PER  | @URU | @BRA | VEN  | COL  | @CHI | @PAR | BOL  | @PER | URU  | BRA  | @VEN | @COL | ARG  |
| URU    | BRA  | @PAR | ARG  | @PER | @COL | ECU  | BOL  | @CHI | @VEN | PAR  | @ARG | PER  | COL  | @ECU | @BOL | CHI  | VEN  | @BRA |
| ARG    | ECU  | @BOL | @URU | VEN  | @CHI | BRA  | COL  | @PER | @PAR | BOL  | URU  | @VEN | CHI  | @BRA | @COL | PER  | PAR  | @ECU |
| PAR    | @CHI | URU  | @ECU | BRA  | BOL  | @VEN | PER  | @COL | ARG  | @URU | ECU  | @BRA | @BOL | VEN  | @PER | COL  | @ARG | CHI  |
| BOL    | @COL | ARG  | @VEN | ECU  | @PAR | CHI  | @URU | BRA  | PER  | @ARG | VEN  | @ECU | PAR  | @CHI | URU  | @BRA | @PER | COL  |

In [9]:
base_path_french_basic = "output/french-basic"

In [10]:
basic_french_model = FootballSchedulerModel(n, SymetricScheme.FRENCH)
write_model_and_raffle(base_path_french_basic, basic_french_model)
assert_optimized(basic_french_model)
print(
    f"Optimization of the basic french model took: {basic_french_model.get_solving_time()} seconds"
)
write_solution(base_path_french_basic, basic_french_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/french-basic/model.lp
Optimization of the basic french model took: 2.797029 seconds


In [11]:
write_solution_to_latex(
    base_path_french_basic, recover_solution_as_df(base_path_french_basic, n)
)

## English Scheme

In [12]:
base_path_english = "output/english"

In [13]:
english_model = FootballSchedulerModel(n, SymetricScheme.ENGLISH, top_teams)
write_model_and_raffle(base_path_english, english_model)
assert_optimized(english_model)
print(
    f"Optimization of the english model with top teams took: {english_model.get_solving_time()} seconds"
)
write_solution(base_path_english, english_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/english/model.lp
Optimization of the english model with top teams took: 3.812664 seconds


In [14]:
english_df = recover_solution_as_mapped_df(base_path_english)
write_solution_to_latex(base_path_english, english_df)
display(Markdown(english_df.to_markdown(index=False)))

| Team   | 0    | 1    | 2    | 3    | 4    | 5    | 6    | 7    | 8    | 9    | 10   | 11   | 12   | 13   | 14   | 15   | 16   | 17   |
|:-------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
| PER    | VEN  | @COL | PAR  | @CHI | @ARG | BRA  | ECU  | @URU | @BOL | BOL  | @VEN | COL  | @PAR | CHI  | ARG  | @BRA | @ECU | URU  |
| CHI    | @URU | BOL  | @ARG | PER  | @COL | PAR  | BRA  | @ECU | @VEN | VEN  | URU  | @BOL | ARG  | @PER | COL  | @PAR | @BRA | ECU  |
| BRA    | COL  | @PAR | @ECU | VEN  | URU  | @PER | @CHI | BOL  | ARG  | @ARG | @COL | PAR  | ECU  | @VEN | @URU | PER  | CHI  | @BOL |
| COL    | @BRA | PER  | @BOL | ECU  | CHI  | @VEN | @ARG | PAR  | @URU | URU  | BRA  | @PER | BOL  | @ECU | @CHI | VEN  | ARG  | @PAR |
| VEN    | @PER | ECU  | URU  | @BRA | @PAR | COL  | @BOL | ARG  | CHI  | @CHI | PER  | @ECU | @URU | BRA  | PAR  | @COL | BOL  | @ARG |
| ECU    | ARG  | @VEN | BRA  | @COL | BOL  | @URU | @PER | CHI  | @PAR | PAR  | @ARG | VEN  | @BRA | COL  | @BOL | URU  | PER  | @CHI |
| URU    | CHI  | @ARG | @VEN | BOL  | @BRA | ECU  | @PAR | PER  | COL  | @COL | @CHI | ARG  | VEN  | @BOL | BRA  | @ECU | PAR  | @PER |
| ARG    | @ECU | URU  | CHI  | @PAR | PER  | @BOL | COL  | @VEN | @BRA | BRA  | ECU  | @URU | @CHI | PAR  | @PER | BOL  | @COL | VEN  |
| PAR    | @BOL | BRA  | @PER | ARG  | VEN  | @CHI | URU  | @COL | ECU  | @ECU | BOL  | @BRA | PER  | @ARG | @VEN | CHI  | @URU | COL  |
| BOL    | PAR  | @CHI | COL  | @URU | @ECU | ARG  | VEN  | @BRA | PER  | @PER | @PAR | CHI  | @COL | URU  | ECU  | @ARG | @VEN | BRA  |

In [15]:
base_path_english_basic = "output/english-basic"

In [16]:
basic_english_model = FootballSchedulerModel(n, SymetricScheme.ENGLISH)
write_model_and_raffle(base_path_english_basic, basic_english_model)
assert_optimized(basic_english_model)
print(
    f"Optimization of the basic english model took: {basic_english_model.get_solving_time()} seconds"
)
write_solution(base_path_english_basic, basic_english_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/english-basic/model.lp
Optimization of the basic english model took: 3.335408 seconds


In [17]:
write_solution_to_latex(
    base_path_english_basic, recover_solution_as_df(base_path_english_basic, n)
)

## Inverted scheme

In [18]:
base_path_inverted = "output/inverted"

In [19]:
inverted_model = FootballSchedulerModel(n, SymetricScheme.INVERTED, top_teams)
write_model_and_raffle(base_path_inverted, inverted_model)
assert_optimized(inverted_model)
print(
    f"Optimization of the inverted model with top teams took: {inverted_model.get_solving_time()} seconds"
)
write_solution(base_path_inverted, inverted_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/inverted/model.lp
Optimization of the inverted model with top teams took: 41.876746 seconds


In [20]:
inverted_df = recover_solution_as_mapped_df(base_path_inverted)
write_solution_to_latex(base_path_inverted, inverted_df)
display(Markdown(inverted_df.to_markdown(index=False)))

| Team   | 0    | 1    | 2    | 3    | 4    | 5    | 6    | 7    | 8    | 9    | 10   | 11   | 12   | 13   | 14   | 15   | 16   | 17   |
|:-------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
| PER    | PAR  | @BOL | @ECU | URU  | @COL | BRA  | CHI  | @ARG | @VEN | VEN  | ARG  | @CHI | @BRA | COL  | @URU | ECU  | BOL  | @PAR |
| CHI    | ECU  | @COL | ARG  | @BRA | @VEN | BOL  | @PER | PAR  | @URU | URU  | @PAR | PER  | @BOL | VEN  | BRA  | @ARG | COL  | @ECU |
| BRA    | VEN  | @ECU | @URU | CHI  | PAR  | @PER | @BOL | COL  | @ARG | ARG  | @COL | BOL  | PER  | @PAR | @CHI | URU  | ECU  | @VEN |
| COL    | @URU | CHI  | @VEN | ECU  | PER  | @PAR | ARG  | @BRA | BOL  | @BOL | BRA  | @ARG | PAR  | @PER | @ECU | VEN  | @CHI | URU  |
| VEN    | @BRA | ARG  | COL  | @BOL | CHI  | @URU | @PAR | ECU  | PER  | @PER | @ECU | PAR  | URU  | @CHI | BOL  | @COL | @ARG | BRA  |
| ECU    | @CHI | BRA  | PER  | @COL | @BOL | ARG  | URU  | @VEN | PAR  | @PAR | VEN  | @URU | @ARG | BOL  | COL  | @PER | @BRA | CHI  |
| URU    | COL  | @PAR | BRA  | @PER | @ARG | VEN  | @ECU | BOL  | CHI  | @CHI | @BOL | ECU  | @VEN | ARG  | PER  | @BRA | PAR  | @COL |
| ARG    | BOL  | @VEN | @CHI | PAR  | URU  | @ECU | @COL | PER  | BRA  | @BRA | @PER | COL  | ECU  | @URU | @PAR | CHI  | VEN  | @BOL |
| PAR    | @PER | URU  | BOL  | @ARG | @BRA | COL  | VEN  | @CHI | @ECU | ECU  | CHI  | @VEN | @COL | BRA  | ARG  | @BOL | @URU | PER  |
| BOL    | @ARG | PER  | @PAR | VEN  | ECU  | @CHI | BRA  | @URU | @COL | COL  | URU  | @BRA | CHI  | @ECU | @VEN | PAR  | @PER | ARG  |

In [21]:
base_path_inverted_basic = "output/inverted-basic"

In [22]:
basic_inverted_model = FootballSchedulerModel(n, SymetricScheme.INVERTED, top_teams)
write_model_and_raffle(base_path_inverted_basic, basic_inverted_model)
assert_optimized(basic_inverted_model)
print(
    f"Optimization of the basic inverted model took: {basic_inverted_model.get_solving_time()} seconds"
)
write_solution(base_path_inverted_basic, basic_inverted_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/inverted-basic/model.lp
Optimization of the basic inverted model took: 43.258383 seconds


In [23]:
write_solution_to_latex(
    base_path_inverted_basic, recover_solution_as_df(base_path_inverted_basic, n)
)

## Back to Back scheme

In [24]:
base_path_b2b = "output/b2b"

In [25]:
b2b_model = FootballSchedulerModel(n, SymetricScheme.BACK_TO_BACK, top_teams)
write_model_and_raffle(base_path_b2b, b2b_model)
assert_optimized(b2b_model)
print(
    f"Optimization of the b2b model with top teams took: {b2b_model.get_solving_time()} seconds"
)
write_solution(base_path_b2b, b2b_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/b2b/model.lp
Optimization of the b2b model with top teams took: 0.20366499999999998 seconds


In [26]:
b2b_df = recover_solution_as_mapped_df(base_path_b2b)
write_solution_to_latex(base_path_b2b, b2b_df)
display(Markdown(b2b_df.to_markdown(index=False)))

| Team   | 0    | 1    | 2    | 3    | 4    | 5    | 6    | 7    | 8    | 9    | 10   | 11   | 12   | 13   | 14   | 15   | 16   | 17   |
|:-------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
| PER    | @PAR | PAR  | COL  | @COL | @CHI | CHI  | ARG  | @ARG | @BOL | BOL  | VEN  | @VEN | URU  | @URU | @ECU | ECU  | BRA  | @BRA |
| CHI    | @ARG | ARG  | @ECU | ECU  | PER  | @PER | @COL | COL  | PAR  | @PAR | @BRA | BRA  | BOL  | @BOL | URU  | @URU | VEN  | @VEN |
| BRA    | URU  | @URU | BOL  | @BOL | ECU  | @ECU | @VEN | VEN  | @COL | COL  | CHI  | @CHI | @PAR | PAR  | @ARG | ARG  | @PER | PER  |
| COL    | @ECU | ECU  | @PER | PER  | ARG  | @ARG | CHI  | @CHI | BRA  | @BRA | URU  | @URU | VEN  | @VEN | @BOL | BOL  | @PAR | PAR  |
| VEN    | @BOL | BOL  | ARG  | @ARG | @URU | URU  | BRA  | @BRA | ECU  | @ECU | @PER | PER  | @COL | COL  | PAR  | @PAR | @CHI | CHI  |
| ECU    | COL  | @COL | CHI  | @CHI | @BRA | BRA  | PAR  | @PAR | @VEN | VEN  | BOL  | @BOL | @ARG | ARG  | PER  | @PER | @URU | URU  |
| URU    | @BRA | BRA  | PAR  | @PAR | VEN  | @VEN | @BOL | BOL  | ARG  | @ARG | @COL | COL  | @PER | PER  | @CHI | CHI  | ECU  | @ECU |
| ARG    | CHI  | @CHI | @VEN | VEN  | @COL | COL  | @PER | PER  | @URU | URU  | PAR  | @PAR | ECU  | @ECU | BRA  | @BRA | @BOL | BOL  |
| PAR    | PER  | @PER | @URU | URU  | BOL  | @BOL | @ECU | ECU  | @CHI | CHI  | @ARG | ARG  | BRA  | @BRA | @VEN | VEN  | COL  | @COL |
| BOL    | VEN  | @VEN | @BRA | BRA  | @PAR | PAR  | URU  | @URU | PER  | @PER | @ECU | ECU  | @CHI | CHI  | COL  | @COL | ARG  | @ARG |

In [27]:
base_path_b2b_basic = "output/b2b-basic"

In [28]:
basic_b2b_model = FootballSchedulerModel(n, SymetricScheme.BACK_TO_BACK, top_teams)
write_model_and_raffle(base_path_b2b_basic, basic_b2b_model)
assert_optimized(basic_b2b_model)
print(
    f"Optimization of the basic b2b model took: {basic_b2b_model.get_solving_time()} seconds"
)
write_solution(base_path_b2b_basic, basic_b2b_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/b2b-basic/model.lp
Optimization of the basic b2b model took: 0.19777599999999998 seconds


In [29]:
write_solution_to_latex(
    base_path_b2b_basic, recover_solution_as_df(base_path_b2b_basic, n)
)

## References:
- [1] G. Durán, E. Mijangos, and M. Frisk, “Scheduling the South American qualifiers to the 2018 FIFA World Cup by integer programming,” European Journal of Operational Research, vol. 262, no. 3, pp. 1035–1048, 2017.
- [2] Pyscipopt Docs.